In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
%matplotlib inline

df =  pd.read_csv('german_credit_data.csv')
print(df.head())

X = df.drop('Risk',axis=1)
Y = df['Risk']
X = pd.get_dummies(X,drop_first=True)
Y = pd.get_dummies(Y,drop_first=True)

for col in X.columns:
    X[col] = X.apply(lambda rec : 1 if rec[col]==True else rec[col], axis=1) 
    X[col] = X.apply(lambda rec : 0 if rec[col]==False else rec[col], axis=1)
for col in Y.columns:
    Y[col] = Y.apply(lambda rec : 1 if rec[col]==True else rec[col], axis=1) 
    Y[col] = Y.apply(lambda rec : 0 if rec[col]==False else rec[col], axis=1)

X = sm.add_constant(X)

   Unnamed: 0  Age     Sex  Job Housing Saving accounts Checking account  \
0           0   67    male    2     own             NaN           little   
1           1   22  female    2     own          little         moderate   
2           2   49    male    1     own          little              NaN   
3           3   45    male    2    free          little           little   
4           4   53    male    2    free          little           little   

   Credit amount  Duration              Purpose  Risk  
0           1169         6             radio/TV  good  
1           5951        48             radio/TV   bad  
2           2096        12            education  good  
3           7882        42  furniture/equipment  good  
4           4870        24                  car   bad  


In [64]:
df.value_counts('Risk')

Risk
good    700
bad     300
Name: count, dtype: int64

In [65]:
train_x , test_x , train_y, test_y = train_test_split(X,Y,train_size=0.7, random_state=42)

In [66]:
"""
    REMEMBER: 
"""

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

tree_gini = DecisionTreeClassifier(criterion='gini').fit(train_x,train_y)

In [67]:
pred_df_gini = pd.DataFrame()
pred_df_gini['pred_good']= tree_gini.predict(test_x)
pred_df_gini['actual_good'] = np.array(test_y)

In [68]:
print(roc_auc_score(pred_df_gini['actual_good'],pred_df_gini['pred_good']))

0.5277617119722382


In [69]:
# entropy based
tree_gini = DecisionTreeClassifier(criterion='entropy', max_depth=3).fit(train_x,train_y)

pred_df_ent = pd.DataFrame()
pred_df_ent['pred_good']= tree_gini.predict(test_x)
pred_df_ent['actual_good'] = np.array(test_y)

print(roc_auc_score(pred_df_ent['actual_good'],pred_df_ent['pred_good']))

0.5482149429517851


In [70]:
# using gridsearch
from sklearn.model_selection import GridSearchCV

tuned_params = [{'criterion': ['gini','entropy'], 'max_depth': range(2,10)}]
clf = GridSearchCV(DecisionTreeClassifier(), tuned_params , cv = 10, scoring='roc_auc')
clf.fit(train_x,train_y)
print(clf.best_score_)
print(clf.best_params_)

0.6108945578231293
{'criterion': 'gini', 'max_depth': 6}
